In [1]:
#!/usr/local/bin/python3
# solver2021.py : 2021 Sliding tile puzzle solver
#
# Code by: name IU ID
#
# Based on skeleton code by D. Crandall & B551 Staff, September 2021
#

In [2]:
import sys
import numpy as np
from time import time

# Housekeeping functions

In [3]:
ROWS=5
COLS=5

In [4]:
def printable_board(board):
    return [ ('%3d ')*COLS  % board[j:(j+COLS)] for j in range(0, ROWS*COLS, COLS) ]

In [5]:
def parse_board(filename):

    start_state = []
    with open(filename, 'r') as file:
        for line in file:
            start_state += [int(i) for i in line.split()]
        return np.reshape(start_state, (5,5))
            
        

In [6]:
board1 = np.array(parse_board('board1.txt'))
board1

array([[12,  6, 10,  1, 23],
       [16, 17, 13,  7, 25],
       [19, 14,  8,  3,  4],
       [21, 15,  9, 20,  5],
       [22, 18, 24,  2, 11]])

In [7]:
board05 = np.array(parse_board('board0.5.txt'))
board05

array([[ 6, 21,  3,  4, 10],
       [ 7,  2,  8,  9, 11],
       [16, 13, 19, 14, 20],
       [25, 12, 18, 15, 24],
       [ 1, 17, 22, 23,  5]])

In [8]:
board0 = np.array(parse_board('board0.txt'))
board0

array([[ 2, 23,  4,  5, 10],
       [ 1,  7,  3,  9, 11],
       [ 6, 13,  8, 15, 20],
       [12, 17, 14, 19, 25],
       [16, 21, 22, 18, 24]])

In [9]:
goal_state = np.array([x for x in range(1,26)]).reshape(5,5)
goal_state

array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25]])

In [10]:
# check if we've reached the goal
def is_goal(state):
    goal_state = np.array([x for x in range(1,26)]).reshape(5,5)

    return np.all(state==goal_state)

In [11]:
def path_trace_back(goal):

    path =[]
    path.append(goal[2])
    step = goal

    # sequence of moves generated by successor function
    moves_list = [x for x in successors(board)[:,1]]

    # index to find inverse move given a certain move, e.g., L1 --> R1
    inverse_moves_idx = [5,6,7,8,9,0,1,2,3,4,15,16,17,18,19,10,11,12,13,14,21,20,23,22]

    while step[1]>0:
        # find current move in the moves list generated from successor function
        inverse_move = moves_list.index(step[2])


        # find index of inverse of current move
        succ_idx = inverse_moves_idx[inverse_move]

        # use successor fucntion to find previous state
        prev_state = successors(step[0])[succ_idx][0]

        # find possible previous steps from visited list
        prev_idx = np.where(visited[:,1] == step[1]-1)

        #index of previous state from possibles
        possible_prev = visited[prev_idx]
        # find match for board in visited list
        step_back_idx = [np.all(possible_prev[n][0] == prev_state) for n in range(len(possible_prev))].index(1)
        step_back = possible_prev[step_back_idx]

        path.append(step_back[2])

        step = step_back
    path.pop()
    path.reverse()

    return path

# Successor function

In [12]:
# return a list of possible successor states
def successors(state):
    
    ## outputs a 24 X 5 X 5 numpy array containing all possible board configs from current config##
    
    #left moves
    L1 = np.vstack([np.roll(state[0], -1),state[1:]])
    L2 = np.vstack([state[:1], np.roll(state[1],-1), state[2:]])
    L3 = np.vstack([state[:2], np.roll(state[2],-1), state[3:]])
    L4 = np.vstack([state[:3], np.roll(state[3],-1), state[4:]])
    L5 = np.vstack([state[:4], np.roll(state[4],-1)])

    #right moves
    R1 = np.vstack([np.roll(state[0],1),state[1:]])
    R2 = np.vstack([state[:1], np.roll(state[1],1), state[2:]])
    R3 = np.vstack([state[:2], np.roll(state[2],1), state[3:]])
    R4 = np.vstack([state[:3], np.roll(state[3],1), state[4:]])
    R5 = np.vstack([state[:4], np.roll(state[4],1)])

    #down moves
    D1 = np.vstack([np.roll(state[:,0],1),state[:,1:].T]).T
    D2 = np.vstack([state[:,0].T, np.roll(state[:,1],1), state[:,2:].T]).T
    D3 = np.vstack([state[:,:2].T, np.roll(state[:,2],1), state[:,3:].T]).T
    D4 = np.vstack([state[:,:3].T, np.roll(state[:,3],1), state[:,4:].T]).T
    D5 = np.vstack([state[:,:4].T, np.roll(state[:,4],1), state[:,5:].T]).T

    #up moves
    U1 = np.vstack([np.roll(state[:,0],-1),state[:,1:].T]).T
    U2 = np.vstack([state[:,0].T, np.roll(state[:,1],-1), state[:,2:].T]).T
    U3 = np.vstack([state[:,:2].T, np.roll(state[:,2],-1), state[:,3:].T]).T
    U4 = np.vstack([state[:,:3].T, np.roll(state[:,3],-1), state[:,4:].T]).T
    U5 = np.vstack([state[:,:4].T, np.roll(state[:,4],-1), state[:,5:].T]).T
    
    #ring moves
    flat_board = np.reshape(state, (1,25))

    #outer moves
    oc_ind = np.array([5,0,1,2,3,10,6,7,8,4,15,11,12,13,9,20,16,17,18,14,21,22,23,24,19])
    Oc = np.reshape(flat_board[:,oc_ind],(5,5))

    occ_ind = np.array([1,2,3,4,9,0,6,7,8,14,5,11,12,13,19,10,16,17,18,24,15,20,21,22,23])
    Occ = np.reshape(flat_board[:,occ_ind],(5,5))


    #inner moves
    ic_ind = np.array([0,1,2,3,4,5,11,6,7,9,10,16,12,8,14,15,17,18,13,19,20,21,22,23,24])
    Ic = np.reshape(flat_board[:,ic_ind],(5,5))

    icc_ind = np.array([0,1,2,3,4,5,7,8,13,9,10,6,12,18,14,15,11,16,17,19,20,21,22,23,24])
    Icc = np.reshape(flat_board[:,icc_ind],(5,5))
    
    return np.array([(L1,'L1'),(L2,'L2'),(L3,'L3'),(L4,'L4'),(L5,'L5'),
                     (R1,'R1'),(R2,'R2'),(R3,'R3'),(R4,'R4'),(R5,'R5'),
                     (U1,'U1'),(U2, 'U2'),(U3,'U3'),(U4,'U4'),(U5,'U5'),
                     (D1,'D1'),(D2,'D2'),(D3,'D3'),(D4,'D4'),(D5,'D5'),
                     (Oc,'Oc'),(Occ,'Occ'),(Ic,'Ic'),(Icc,'Icc')], dtype = object)
    

# heuristic functions:

In [13]:
#average number of tiles moved for all possible moves
n_tiles = [25- np.count_nonzero(successors(goal_state)[:,0][i]==goal_state) for i in range(len(successors(goal_state)))]
avg_tiles = np.mean(n_tiles)
avg_tiles

6.166666666666667

In [14]:
def sum_abs_diff(state):
    return np.sum(np.abs(np.subtract(state,goal_state)))

In [15]:
def avg_abs_diff(state):
    abs_diff = np.sum(np.abs(np.subtract(state, goal_state)))
    n_displaced = 25 - np.count_nonzero(state == goal_state)
    return int(abs_diff/n_displaced)



In [16]:
def n_displaced_outer(state):
    
    outer_val = [1,2,3,4,5,6,10,11,15,16,20,21,22,23,24,25]
    outer_idx = [int(np.where(np.ndarray.flatten(goal_state== x))[0]) for x in outer_val]

    return len(outer_idx)-np.count_nonzero((np.ndarray.flatten(state)==np.ndarray.flatten(goal_state))[outer_idx])

In [17]:
def n_displaced_inner(state):
    
    inner_val = [7,8,9,12,14,17,18,19]
    inner_idx = [int(np.where(np.ndarray.flatten(goal_state== x))[0]) for x in inner_val]

    return len(inner_idx)-np.count_nonzero((np.ndarray.flatten(state)==np.ndarray.flatten(goal_state))[inner_idx])


In [18]:
def n_outer_minus_inner(state):
    return n_displaced_outer(state) - n_displaced_inner(state)

In [19]:
def min_manhattan(number):

    #row and col indices for goal state
    goal_idx = [np.where(goal_state == i) for i in range(1,26)]
    goal_idx_dict = {}
    for i, coord in enumerate(goal_idx, 1):
        goal_idx_dict[i] = (coord)

    #rol, col indices for a number on any board
    board_row , board_col = np.where(board == number)

    #number or rows and columns from current to goal
    coord_off = np.subtract(np.where(board == number), goal_idx_dict[number])

    #limit distance to 2
    max_off = [0,1,2,2,1]
    row_off = max_off[int(coord_off[0])]
    col_off = max_off[int(coord_off[1])]

    return row_off+col_off

In [20]:
def sum_min_manhattan(board):
    return np.sum([min_manhattan(n) for n in board.reshape(25)])

In [21]:
def n_displaced_tiles(state):
    compare = state == goal_state
    return 25 - np.count_nonzero(compare)

In [22]:
def manhattan_displaced(state):
    return sum_min_manhattan(state)/n_displaced_tiles(state)

In [23]:
def n_out_of_row(board):

    correct_in_row = []
    correct_in_row.append([x in board[0,:] for x in [1,2,3,4,5]])
    correct_in_row.append([x in board[1,:] for x in [6,7,8,9,10]])
    correct_in_row.append([x in board[2,:] for x in [11,12,13,14,15]])
    correct_in_row.append([x in board[3,:] for x in [16,17,18,19,20]])
    correct_in_row.append([x in board[4,:] for x in [21,22,23,24,25]])

    return 25-np.count_nonzero(np.array(correct_in_row).reshape(1,25))


# easy board for testing

In [24]:
# test to see if flow of solve function works
# easy board is solved in 1 move
easy_board = successors(goal_state)[0][0]
easy_board

array([[ 2,  3,  4,  5,  1],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25]])

In [25]:
successors(easy_board)[5][0]

array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25]])

In [26]:
np.all(successors(easy_board)[5][0]==goal_state)

True

# Solves Board 0.5

In [24]:
time_0=time()
#initialize fringe
initial_board = board05
fringe = (initial_board,0,'staring point', 0)# (board, g, move, f)
fringe = np.array(fringe, dtype = object).reshape(1,4)

#create list to keep track of items from fringe that are checked
visited = np.zeros(4).reshape(1,4)

while len(fringe)>0:
#     found = False
#     while not found:
    pop_idx = np.argmin(fringe[:,3])
    pop_state = fringe[np.argmin(fringe[:,3])].reshape(1,4)

    visited = np.append(visited, pop_state).reshape(-1,4)
    fringe = np.delete(fringe, pop_idx, axis =0)
    state = pop_state[0][0]
    g = pop_state[0][1]
    move = pop_state[0][2]
    f = pop_state[0][3]

    

    if is_goal(state):
        goal = visited[-1]
        print('goal state found')
        print(state)
        print (f'solve found in {g} moves')
        print(str(path_trace_back(goal)))
        found = True
        break

    else:
        for board, move in successors(state):   
            h = n_out_of_row(board)
            f= g+1+h
            fringe = np.append(fringe, (board, g+1, move, f)).reshape(-1,4)

print(f'solution found in {np.round((time()-time_0)/60,3)} mins')

C:\Users\joyrl\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


goal state found
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]]
solve found in 7 moves
['D5', 'U2', 'D1', 'Icc', 'R3', 'Ic', 'L5']
solution found in 0.015 mins


# Try to solve Board1
addtion to above:  randomly chooses items from fringe that have the same (minimum value)

In [70]:
time_0=time()
#initialize fringe
initial_board = board1
fringe = (initial_board,0,'staring point', 0)# (board, g, move, f)
fringe = np.array(fringe, dtype = object).reshape(1,4)

#create list to keep track of items from fringe that are checked
visited = np.zeros(4).reshape(1,4)


while len(fringe)>0:
    # find items in fringe with min f value and pick a random on to pop off
    min_fs =np.where(fringe[:,3]==fringe[:,3].min())
    rand_f_idx = np.random.choice(min_fs[0],1)
    rand_min_f = fringe[rand_f_idx]
    pop_state = rand_min_f.reshape(1,4)
    
    visited = np.append(visited, pop_state).reshape(-1,4)
    fringe = np.delete(fringe, rand_f_idx, axis =0)
    
    # make some variable assignments
    state = pop_state[0][0]
    g = pop_state[0][1]
    move = pop_state[0][2]
    f = pop_state[0][3]
    
    # check if state is goal
    if is_goal(state):
        goal = visited[-1]
        print('goal state found')
        print(state)
        print (f'solve found in {g} moves')
        print(str(path_trace_back(goal)))
        found = True
        break
        
    
    # if not goal, calculate f and add to fringe
    else:
        for board, move in successors(state):   
            h = n_out_of_row(board)
            f= g+1+h
            fringe = np.append(fringe, (board, g+1, move, f)).reshape(-1,4)

print(f'solution found in {np.round((time()-time_0)/60,3)} mins')

KeyboardInterrupt: 